<div style="border: 1px solid black">
<b><center><font size="3">Data Acquisition</font></center></b>

<b><center><font size="2">Web Scraping Project</font></center></b>
</div>

**Notebook Developed by**: Rene Jerez </b>

**Notebook Co-writer by**: ChatGPT </b>

**Notebook Based on**: [Ricardo Campos](https://www.di.ubi.pt/~rcampos)<br>



<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span>
<a href="#Informação de uma página web estática 1" data-toc-modified-id="Informação de uma página web estática 1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Informação de uma página web estática</a></span></span></li>
<li><span>
<a href="#Informação de uma página web dinâmica 2" data-toc-modified-id="Informação de uma página web dinâmica 2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Informação de uma página web dinâmica</a></span>


### 1. Informação de uma página web estática

Obter melhores preços de tickets aereos da Companhia RyanAir com origem no Aeroporto do Porto

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlencode
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import logging

In [16]:
# logging settings
logging.basicConfig(level=logging.INFO)

In [21]:
########################################
# Make a URL variable
########################################

# Base URL
base_url = 'https://www.ryanair.com/gb/en/cheap-flights-beta'

# Query parameters
payload = {
    'originIata': 'OPO',
    'destinationIata': 'ANY',
    'isReturn': 'true',
    'isMacDestination': 'false',
    'promoCode': '',
    'adults': '1',
    'teens': '0',
    'children': '0',
    'infants': '0',
    'dateOut': '2024-04-01',
    'dateIn': '2024-04-30',
    'daysTrip': '3',
    'dayOfWeek': 'FRIDAY',
    'isExactDate': 'false',
    'outboundFromHour': '00:00',
    'outboundToHour': '23:59',
    'inboundFromHour': '00:00',
    'inboundToHour': '23:59',
    'priceValueTo': '',
    'currency': 'EUR',
    'isFlexibleDay': 'false'
}

# Encode parameters
query_string = urlencode(payload)

# Full URL
ryanair_url = f"{base_url}?{query_string}"
#ryanair_url = 'https://www.ryanair.com/gb/en/cheap-flights-beta?originIata=OPO&destinationIata=ANY&isReturn=false&isMacDestination=false&promoCode=&adults=1&teens=0&children=0&infants=0&dateOut=2024-04-01&dateIn=2024-04-30&daysTrip=3&dayOfWeek=FRIDAY&isExactDate=false&outboundFromHour=00:00&outboundToHour=23:59&priceValueTo=&currency=EUR&isFlexibleDay=false'


In [22]:
########################################
# Selenium Exraction
########################################

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open URL
driver.get(ryanair_url)

# Process info
logging.info("Start extracting content")

# Wait
time.sleep(60)

# Extract info
page_content = driver.page_source


# Save info in html file
with open('page.html', 'w', encoding='utf-8') as f:
    f.write(page_content)

driver.quit()

2024-04-02 22:45:15,864 - INFO - ====== WebDriver manager ======
2024-04-02 22:45:17,068 - INFO - Get LATEST chromedriver version for google-chrome
2024-04-02 22:45:17,902 - INFO - Get LATEST chromedriver version for google-chrome
2024-04-02 22:45:18,913 - INFO - Driver [C:\Users\rene_\.wdm\drivers\chromedriver\win64\123.0.6312.86\chromedriver-win32/chromedriver.exe] found in cache
2024-04-02 22:46:26,954 - INFO - Start extracting content


In [23]:
########################################
# Find destinations in the full content
########################################

# Load HTML file
with open('page.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse with Soup
soup = BeautifulSoup(html_content, 'html.parser')

# Find destinations classes
elements = soup.find_all(class_='country-card__content')
all_elements_html = ""


# Check if any elements were found
if elements:
    for element in elements:
        # Convert to a string
        element_html = str(element)

        # Append all elements
        all_elements_html += element_html + "\n\n"
        
else:
    print("No destinations classes found.")


# Save file 'destinations.html'
if all_elements_html:
    with open('destinations.html', 'w', encoding='utf-8') as f:
        f.write(all_elements_html)
    print("File 'destinations.html' has been created with the extracted content.")
    
else:
    print("No content to write to 'destinations.html'.")

File 'destinations.html' has been created with the extracted content.


In [26]:
########################################
# Convert from HTML to Dataframe
########################################

# Load HTML file
with open('destinations.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse HTML with Soup
soup = BeautifulSoup(html_content, 'html.parser')

# Info we need
countries = []
dates = []
prices = []

# Find destinations
elements = soup.find_all(class_='country-card__content')

# Select informations about Country, Date and Price
for element in elements:
    # Country
    country = element.find(class_='country-card__country').text.strip()
    countries.append(country)
    
    # Date
    date_info = element.find(class_='b2').text.strip()
    dates.append(date_info)
    
    # Price
    price = element.find(class_='country-card__price').text.strip()
    prices.append(price)

# Final dataframe
df_ryanair = pd.DataFrame({
    'Country': countries,
    'Date Info': dates,
    'Price': prices
})

In [27]:
########################################
# Flights Recommendations
########################################
df_ryanair

,Country,Date Info,Price
0,Portugal 3 days in April,3 days in April,€94.57
1,Morocco 3 days in April,3 days in April,€105.04
2,Spain 3 days in April,3 days in April,€122.88
3,United Kingdom 3 days in April,3 days in April,€123.88
4,France 3 days in April,3 days in April,€171.37
5,Hungary 3 days in April,3 days in April,€191.62
6,Belgium 3 days in April,3 days in April,€239.30


### 2. Informação de uma página web Dinâmica

Obter Contratos Publicos recentes de origem da Covilhã

In [8]:
########################################
# Make a URL variable
########################################

# Base URL
base_url = 'https://www.base.gov.pt/Base4/pt/pesquisa/'

# Query parameters
payload = {
            'type':'contratos',
            'texto':'',
            'tipo':'0',
            'tipocontrato':'0',
            'cpv':'',
            'aqinfo':'',
            'adjudicante':'',
            'adjudicataria':'',
            'sel_price':'price_c1',
            'desdeprecocontrato':'',
            'ateprecocontrato':'',
            'desdeprecoefectivo':'',
            'ateprecoefectivo':'',
            'desdeprazoexecucao':'',
            'ateprazoexecucao':'',
            'sel_date':'date_c1',
            'desdedatacontrato':'2024-03-01',
            'atedatacontrato':'2024-03-31',
            'desdedatapublicacao':'',
            'atedatapublicacao':'',
            'desdedatafecho':'',
            'atedatafecho':'',
            'pais':'187',
            'distrito':'6',
            'concelho':'68'
}

# Encode parameters
query_string = urlencode(payload)

# Full URL
url_covilha = f"{base_url}?{query_string}"


In [17]:
########################################
# Extract content from website
########################################

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open URL
driver.get(url_covilha)

# Process info
logging.info("Start extracting content")

# Wait
time.sleep(60)

# Locate the table
table = driver.find_element(By.CLASS_NAME, 'table-striped')

# Assuming headers are in the first row
headers = [th.text for th in table.find_elements(By.TAG_NAME, "th")]
logging.info("Set headers")

overall_dict = {}
for index, row in enumerate(table.find_elements(By.TAG_NAME, "tr")[1:], start=1):
    cells = row.find_elements(By.TAG_NAME, "td")
    row_dict = {headers[i]: cell.text for i, cell in enumerate(cells)}
    overall_dict[f"row_{index}"] = row_dict

logging.info("Finish extraction")

# Close the browser
driver.quit()

2024-04-02 22:36:50,887 - INFO - ====== WebDriver manager ======
2024-04-02 22:36:52,133 - INFO - Get LATEST chromedriver version for google-chrome
2024-04-02 22:36:53,516 - INFO - Get LATEST chromedriver version for google-chrome
2024-04-02 22:36:54,492 - INFO - Driver [C:\Users\rene_\.wdm\drivers\chromedriver\win64\123.0.6312.86\chromedriver-win32/chromedriver.exe] found in cache
2024-04-02 22:37:04,212 - INFO - Start extracting content
2024-04-02 22:38:04,381 - INFO - Set headers
2024-04-02 22:38:10,603 - INFO - Finish extraction


In [18]:
########################################
# Convert from dictionary to dataframe
########################################
df_rows = pd.DataFrame.from_dict(overall_dict, orient='index')

In [19]:
df_rows.head()

,Objeto do contrato,Tipo de procedimento,Adjudicante,Adjudicatário,Preço contratual,Publicação,
row_1,Consulta Prévia nº 02/SASUBI/2024 - Aquisição ...,Consulta Prévia,Serviços de Acção Social da Universidade da Be...,PANIFICADORA CENTRAL DO TORTOSENDO LDA,"22.977,80 €",02-04-2024,
row_2,Ajuste Direto nº 02/SASUBI/2024 - Aquisição de...,Ajuste Direto Regime Geral,Serviços de Acção Social da Universidade da Be...,CONTACTO - REPRESENTAÇÕES E COMÉRCIO DE BEBIDA...,"8.986,44 €",02-04-2024,
row_3,Produtos de higiene,Ajuste Direto Regime Geral,Centro de Formação Profissional da Indústria T...,"MUNDISAN - PAPEIS E PRODUTOS DE HIGIENE, LDA","3.588,26 €",02-04-2024,
row_4,Aquisição de Serviços de Reparação de Telhado ...,Ajuste Direto Regime Geral,Escola Profissional Agrícola Quinta da Lageosa...,"António Ventura, Unipessoal, Lda.","760,00 €",02-04-2024,
row_5,Aquisição de dispositivos de proteção individual,Concurso público,"Centro Hospitalar Universitário Cova da Beira,...","BARNA IMPORT MEDICA PORTUGAL-UNIPESSOAL, LDA","24.459,27 €",28-03-2024,
